This document contains the implementation of EC and max EC for color-based filtrations. We then compute them on the first 10 pairs of graphs loaded with the BREC dataset in basic.npy.

The runtime has been re-run after cleaning up the original file (so there are some minor fluctuations in runtime)

In [ ]:
import torch
import itertools
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
import time

In [ ]:
# This block contains the implementations for EC and max EC.

# Function for EC of a graph G = (V, E) with vertex-level filtrations, where V = {0, 1, ..., n}. The algorithm takes in 3 parameters:

# filtration_steps - the filtration values for the vertex based filtration (which is assumed to be ordered)
# filtered_v - the list of values assigned to the vertices of the graph G, the i-th index of the list represents the value on the vertex i
# edge_index - list of edges in G of the form (i, j) for edge between vertex i and j in G.
def make_EC_v(filtration_steps, filtered_v, edge_index):
    filtered_v_torch = torch.Tensor(filtered_v)
    filtered_e_torch, _ = torch.max(torch.stack((filtered_v_torch[edge_index[0]], filtered_v_torch[edge_index[1]])), axis=0)

    v_count = {}
    e_count = {}

    for a_i in filtration_steps:
      v_count[a_i] = 0
      e_count[a_i] = 0

    for val in filtered_v_torch:
      val1 = val.item()
      v_count[val1] += 1

    for val in filtered_e_torch:
      val1 = val.item()
      e_count[val1] += 1

    prev_v = 0
    prev_e = 0
    ec_list = []
    for i in range(0, len(filtration_steps)):
      a_i = filtration_steps[i]
      v_count[a_i] += prev_v
      e_count[a_i] += prev_e
      ec_list.append(v_count[a_i] - e_count[a_i])
      prev_v = v_count[a_i]
      prev_e = e_count[a_i]

    return ec_list, filtered_e_torch.tolist()

# Function for EC of a graph G = (V, E) with edge-level filtrations, where V = {0, 1, ..., n}. The algorithm takes in 3 parameters:

# filtration_steps - the filtration values for the edge based filtration (which is assumed to be ordered list)
# filtered_e - the list of values assigned to the edges of the graph G, the i-th index of the list represents the value on the i-th edge in edge_index
# v_count - the number of vertices of the graph G
def make_EC_e(filtration_steps, filtered_e, v_count):
  # initalize the list with the number of vertices
  ec_list = [v_count]
  e_count = {}

  for a_i in filtration_steps:
    e_count[a_i] = 0
  for val in filtered_e:
    e_count[val] += 1

  for i in range(0, len(filtration_steps)):
    a_i = filtration_steps[i]
    ec_list.append(ec_list[i] - e_count[a_i])

  return ec_list

# Computes the EC diagrams of a graph G = (V, E) with a specified filtrations (fv, fe). The algorithm takes in 5 parameters:

# filtration_v - the list of values assigned to the vertices of the graph G, the i-th index of the list represents the value on the vertex i
# filtration_e - the list of values assigned to the edges of the graph G, the i-th index of the list represents the value on the i-th edge in edge_index
# edge_index - list of edges in G of the form (i, j) for edge between vertex i and j in G.
# filtration_steps_v - the ordered list of filtration values for fv
# filtration_steps_e - the ordered list of filtration values for fe
def EC_diagram(filtration_v, filtration_e, edge_index, filtration_steps_v, filtration_steps_e):
  output_v, _ = make_EC_v(filtration_steps_v, filtration_v, edge_index)
  output_e = make_EC_e(filtration_steps_e, filtration_e, len(filtration_v))
  return [output_v, output_e]

# Computes the max EC diagram of a graph G = (V, E) with a specified filtration fv. The algorithm takes in 3 parameters:

# filtration_v - the list of values assigned to the vertices of the graph G, the i-th index of the list represents the value on the vertex i
# edge_index - list of edges in G of the form (i, j) for edge between vertex i and j in G.
# filtration_steps_v - the ordered list of filtration values for fv
def max_EC_diagram(filtration_v, edge_index, filtration_steps_v):
    output_v, filtration_e = make_EC_v(filtration_steps_v, filtration_v, edge_index)
    output_e = make_EC_e(filtration_steps_v, filtration_e, len(filtration_v))
    return [output_v, output_e]

In [ ]:
# Loading the dataset
from google.colab import drive
drive.mount('/content/drive')

dataset = []
data = np.load(f'/content/drive/My Drive/Colab Notebooks/datasets/BREC/basic.npy')
for i in range(0, data.size, 2):
  pyg_graph_1 =  nx.from_graph6_bytes(data[i].encode())
  pyg_graph_2 =  nx.from_graph6_bytes(data[i+1].encode())
  dataset.append((pyg_graph_1, pyg_graph_2))

print(len(dataset))

# Converting the entries in dataset to the inputs for the EC diagram and the max EC diagrams.
graph_data_list = []
stop = 10
for item in dataset[0:stop]:
  G, H = item

  G_edge_list = [[], []]
  H_edge_list = [[], []]
  for v0, v1 in G.edges:
    G_edge_list[0].append(v0)
    G_edge_list[1].append(v1)

  for v0, v1 in H.edges:
    H_edge_list[0].append(v0)
    H_edge_list[1].append(v1)

  G_nodes = list(G.nodes)
  H_nodes = list(H.nodes)
  G_edges = list(G.edges)
  H_edges = list(H.edges)

  edge_index_G = torch.Tensor(G_edge_list).long()
  edge_index_H = torch.Tensor(H_edge_list).long()

  # Degree Based Coloring
  G_colors = [G.degree[x] for x in G_nodes]
  H_colors = [H.degree[x] for x in H_nodes]

  # print(G_colors)
  # print(H_colors)

  G_edge_colors = []
  for i in range(len(G_edges)):
    a = G_edges[i][0]
    b = G_edges[i][1]
    color = [G_colors[a], G_colors[b]]
    color.sort()
    G_edge_colors.append((color[0], color[1]))

  H_edge_colors = []
  for i in range(len(H_edges)):
    a = H_edges[i][0]
    b = H_edges[i][1]
    color = [H_colors[a], H_colors[b]]
    color.sort()
    H_edge_colors.append((color[0], color[1]))

  # Coloring Set
  X = list(set(G_colors + H_colors))
  edge_X = list(set(G_edge_colors + H_edge_colors))
  # print(X)
  # print(edge_X)

  G_data = (edge_index_G, G_colors, G_edge_colors)
  H_data = (edge_index_H, H_colors, H_edge_colors)

  data_item = [G_data, H_data, X, edge_X]
  graph_data_list.append(data_item)

len(graph_data_list)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
60


10

In [ ]:
# For a coloring set X, and its edge coloring set X x X/~, generate all possible orders of the colors for both.
def filtration_list_ec(X, edge_X):
  X_val = [i+1 for i in range(0, len(X))]
  X_e_val = [i+1 for i in range(0, len(edge_X))]

  # Generate all possible fv's
  values = itertools.permutations(X_val)
  e_values = itertools.permutations(X_e_val)

  return values, e_values, X_val, X_e_val

# For a coloring set X, generate all possible orders of colors for X.
def filtration_list_max_ec(X):
  X_val = [i+1 for i in range(0, len(X))]
  # Generate all possible fv's
  values = itertools.permutations(X_val)

  return values, X_val

In [ ]:
# Compare the runtime between Max EC vs EC
s = time.time()

# Max EC Testing
# It takes in the two graphs G and H with degrees as the coloring on its vertices
# Then it computes the max EC diagrams of G and H with respect to a possible ordering of the colors as it goes through the possible list of color orderings
# If the two diagrams differ, the program breaks early.
for i in range(len(graph_data_list)):
  item = graph_data_list[i]
  edge_index_G, G_colors, G_edge_colors = item[0]
  edge_index_H, H_colors, H_edge_colors = item[1]
  X = item[2]
  edge_X = item[3]

  node_size_G = len(G_colors)
  node_size_H = len(H_colors)

  fv_list, filtration_steps_v = filtration_list_max_ec(X)
  # ec_G_list = {'V': [], 'E': []}
  # ec_H_list = {'V': [], 'E': []}

  break_early = False

  for elt in fv_list:
    fv = {}
    for j in range(len(elt)):
      fv[X[j]] = elt[j]

    input_v_G = [fv[G_colors[x]] for x in range(0, len(G_colors))]
    input_v_H = [fv[H_colors[x]] for x in range(0, len(H_colors))]

    G_output, input_e_G = make_EC_v(filtration_steps_v, input_v_G, edge_index_G)
    H_output, input_e_H = make_EC_v(filtration_steps_v, input_v_H, edge_index_H)

    if G_output != H_output:
      break_early = True
      print("False")
      break

    G_output_e = make_EC_e(filtration_steps_v, input_e_G, node_size_G)
    H_output_e = make_EC_e(filtration_steps_v, input_e_H, node_size_H)

    if G_output_e != H_output_e:
      break_early = True
      print("False")
      break

  if not break_early:
    print("True")

print(time.time() - s)

True
True
True
True
True
True
True
True
True
True
0.17836856842041016


In [ ]:
t = time.time()

# EC Testing
# It takes in the two graphs G and H with degrees as the coloring on its vertices and an induced coloring on its edges
# Then it computes the max EC diagrams of G and H with respect to a possible ordering of the colors as it goes through the possible list of color orderings (in fv first and then in fe)
# If the two diagrams differ, the program breaks early.
for i in range(len(graph_data_list)):
  item = graph_data_list[i]
  edge_index_G, G_colors, G_edge_colors = item[0]
  edge_index_H, H_colors, H_edge_colors = item[1]
  X = item[2]
  edge_X = item[3]

  node_size_G = len(G_colors)
  node_size_H = len(H_colors)
  # print(len(X))
  # print(len(edge_X))
  fv_list, fe_list, filtration_steps_v, filtration_steps_e = filtration_list_ec(X, edge_X)

  break_early = False
  for elt in fv_list:
    fv = {}
    for j in range(len(elt)):
      fv[X[j]] = elt[j]
    input_v_G = [fv[G_colors[x]] for x in range(0, len(G_colors))]
    input_v_H = [fv[H_colors[x]] for x in range(0, len(H_colors))]
    G_output, _ = make_EC_v(filtration_steps_v, input_v_G, edge_index_G)
    H_output, _ = make_EC_v(filtration_steps_v, input_v_H, edge_index_H)

    if G_output != H_output:
      break_early = True
      print("False")
      break

  for elt in fe_list:
    fe = {}
    for j in range(len(elt)):
      fe[edge_X[j]] = elt[j]

    input_e_G = [fe[G_edge_colors[x]] for x in range(0, len(G_edge_colors))]
    input_e_H = [fe[H_edge_colors[x]] for x in range(0, len(H_edge_colors))]

    G_output= make_EC_e(filtration_steps_e, input_e_G, node_size_G)
    H_output= make_EC_e(filtration_steps_e, input_e_H, node_size_H)

    if G_output != H_output:
      break_early = True
      print("False")
      break

  if not break_early:
    print("True")

print(time.time() - t)

True
True
True
True
True
True
True
True
True
True
69.79122066497803
